In [13]:
import random
import math

import numpy as np
import heapq


In [14]:
def generate_graph(size, density, noise_level, negative_values):
    graph = [[] for _ in range(size)]

    for u in range(size):
        for v in range(size):
            if u == v:
                continue

            if random.random() < density:
                # base positive weight in [1, 10]
                base = 1.0 + 9.0 * random.random()

                # add some noise
                w = base + np.random.normal(0.0, noise_level)

                if negative_values:
                    # sometimes make the edge negative (for Bellman–Ford case)
                    if random.random() < 0.1:
                        w = -abs(w) * 0.5
                else:
                    # guarantee non-negative for Dijkstra
                    if w <= 0:
                        w = 0.1

                graph[u].append((v, float(w)))

    return graph


In [15]:
def dijkstra(graph, start):
    n = len(graph)
    dist = [math.inf] * n
    dist[start] = 0.0

    pq = [(0.0, start)]  # (distance, node)

    while pq:
        current_dist, u = heapq.heappop(pq)
        if current_dist > dist[u]:
            continue

        for v, w in graph[u]:
            new_dist = current_dist + w
            if new_dist < dist[v]:
                dist[v] = new_dist
                heapq.heappush(pq, (new_dist, v))

    return dist


In [16]:
def bellman_ford(graph, start):
    n = len(graph)
    dist = [math.inf] * n
    dist[start] = 0.0

    # build edge list
    edges = []
    for u in range(n):
        for v, w in graph[u]:
            edges.append((u, v, w))

    # relax edges n-1 times
    for _ in range(n - 1):
        updated = False
        for u, v, w in edges:
            if dist[u] != math.inf and dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                updated = True
        if not updated:
            break

    # check negative cycles
    for u, v, w in edges:
        if dist[u] != math.inf and dist[u] + w < dist[v]:
            return None, True

    return dist, False


In [17]:
def solve_shortest_path(size=20,
                        density=0.3,
                        noise_level=0.1,
                        negative_values=False,
                        start=0,
                        end=None):
    """
    Generate ONE graph with the given parameters
    and compute the shortest path from start to end.
    """
    if end is None:
        end = size - 1

    print(f"\n--- New problem ---")
    print(f"size={size}, density={density}, noise={noise_level}, "
          f"negative_values={negative_values}, start={start}, end={end}")

    graph = generate_graph(size, density, noise_level, negative_values)

    if negative_values:
        dist, neg_cycle = bellman_ford(graph, start)
        if neg_cycle:
            print("Result: NEGATIVE CYCLE detected → shortest path undefined.")
            return None
        d = dist[end]
    else:
        dist = dijkstra(graph, start)
        d = dist[end]

    if d == math.inf:
        print("Result: NO PATH from", start, "to", end)
        return None
    else:
        print("Result: shortest distance from", start, "to", end, "=", d)
        return d


In [18]:
# Example 1: small positive graph
solve_shortest_path(size=10, density=0.3, noise_level=0.1, negative_values=False)

# Example 2: small graph with possible negative edges
solve_shortest_path(size=10, density=0.5, noise_level=0.1, negative_values=True)

# Example 3: medium graph
solve_shortest_path(size=50, density=0.2, noise_level=0.5, negative_values=False)

# Example 4: bigger graph (fast because no negative edges)
solve_shortest_path(size=100, density=0.1, noise_level=0.1, negative_values=False)



--- New problem ---
size=10, density=0.3, noise=0.1, negative_values=False, start=0, end=9
Result: shortest distance from 0 to 9 = 3.0374281847352753

--- New problem ---
size=10, density=0.5, noise=0.1, negative_values=True, start=0, end=9
Result: NEGATIVE CYCLE detected → shortest path undefined.

--- New problem ---
size=50, density=0.2, noise=0.5, negative_values=False, start=0, end=49
Result: shortest distance from 0 to 49 = 6.622113250631157

--- New problem ---
size=100, density=0.1, noise=0.1, negative_values=False, start=0, end=99
Result: shortest distance from 0 to 99 = 3.995497189036143


3.995497189036143